<center><b>Extract-Transform-Load  (ETL)<b><center>

In [21]:
import requests as rq
import pdb
import pandas as pd

Para acceder a los recursos de la API, se requiere generar un autokey público en la página: 

https://datosabiertos.enacom.gob.ar/developers/

In [22]:
auto_key = "ZXuPjCTsAAlw8wdgE6mXkm66ZfAi9kO9YXO4WeV3"
url = "http://api.datosabiertos.enacom.gob.ar/api/v2/datasets.json/?auth_key=" + auto_key

In [23]:
# Realizar una solicitud GET a la API
def get_data_api(url):
    response = rq.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Analizar la respuesta JSON
        data = response.json()        
        return data
    else:
        print(f"Error al realizar la solicitud: Código de estado {response.status_code}")
        print(f"Url",url)    


In [24]:
data = get_data_api(url)

La siguiente funcion busca los GUIDs. Globally Unique Identifier (GUID) es un tipo de identificador utilizado en llamadas API para identificar de forma única recursos o entidades.

In [25]:
# Codigo de extraccion guids y tags
datos = []
def api_reader(data):
    try: 
        for line in data: 
            datos.append([line['guid'],line['description'],line['tags']])

    except :
        pdb.set_trace()
        
    df = pd.DataFrame(datos,columns=['guid','description','tags'])
    return df

In [26]:
guids = api_reader(data)
guids.info()
guids.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   guid         48 non-null     object
 1   description  48 non-null     object
 2   tags         48 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


,guid,description,tags
0,PROYE-ADJUD-Y-BENEF-ALCAN,Cantidad de proyectos adjudicados por año y ca...,[]
1,FOMEC-PAGOS-Y-ADJUD-75939,Montos en millones de pesos Pagados y Adjudica...,[]
2,DENUN-Y-RECLA-12878,Estadísticas mensuales sobre denuncias y recla...,"[denuncias, reclamos]"
3,PENET-DEL-SERVI-DE-INTER,Penetración del servicio de Internet fijo por ...,"[internet, banda ancha, penetración]"
4,ACCES-A-INTER-FIJO-51042,Número de accesos al servicio de Internet fijo...,"[internet, banda ancha, provincial, banda ango..."


In [27]:
guids[guids['guid'] == "LISTA-DE-LOCAL-CON-CONEC"]

,guid,description,tags


Se ejecuta un drop duplicates en caso se presenten guids duplicados

In [28]:
guids.drop_duplicates(subset=['guid'],inplace=True)
guids.reset_index(inplace=True,drop=True)
guids.info()
guids.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   guid         48 non-null     object
 1   description  48 non-null     object
 2   tags         48 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


,guid,description,tags
0,PROYE-ADJUD-Y-BENEF-ALCAN,Cantidad de proyectos adjudicados por año y ca...,[]
1,FOMEC-PAGOS-Y-ADJUD-75939,Montos en millones de pesos Pagados y Adjudica...,[]
2,DENUN-Y-RECLA-12878,Estadísticas mensuales sobre denuncias y recla...,"[denuncias, reclamos]"
3,PENET-DEL-SERVI-DE-INTER,Penetración del servicio de Internet fijo por ...,"[internet, banda ancha, penetración]"
4,ACCES-A-INTER-FIJO-51042,Número de accesos al servicio de Internet fijo...,"[internet, banda ancha, provincial, banda ango..."


Buscando tags referentes a internet en los guids encontrados, ya que sera el tópico a evaluar:

In [29]:
mask = guids.tags.apply(lambda x:'internet' in x)
guids = guids[mask]
guids.reset_index(inplace=True,drop=True)
guids.info()
guids

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   guid         7 non-null      object
 1   description  7 non-null      object
 2   tags         7 non-null      object
dtypes: object(3)
memory usage: 300.0+ bytes


,guid,description,tags
0,PENET-DEL-SERVI-DE-INTER,Penetración del servicio de Internet fijo por ...,"[internet, banda ancha, penetración]"
1,ACCES-A-INTER-FIJO-51042,Número de accesos al servicio de Internet fijo...,"[internet, banda ancha, provincial, banda ango..."
2,ACCES-A-INTER-FIJO-POR,Número de accesos al servicio de Internet fijo...,"[internet, banda ancha, provincial]"
3,INGRE-POR-LA-OPERA-DEL,Ingresos trimestrales de los operadores por el...,"[internet, ingresos, banda ancha]"
4,VELOC-PROME-DE-BAJAD-DE,Serie histórica de la velocidad media de bajad...,"[internet, velocidad, banda ancha]"
5,ACCES-A-INTER-FIJO-BANDA,Número de accesos al servicio de Internet fijo...,"[internet, banda ancha, provincial, banda ango..."
6,CONEC-AL-SERVI-DE-INTER,Mapa geolocalizado de las distintas tecnología...,"[internet, telefonia movil, banda ancha, provi..."


In [30]:
guids_name = guids['guid'].to_list()

Adquiriendo la data de los recursos disponibles en la API (GUIDs). Aqui se necesita el autokey publico generado en la API (primer paso).

In [31]:
datos_guid = {}
def guids_reader(guids):
    for guid in guids:
        guid_str = guid
        guid_url = "http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/" + guid_str + "/data.ajson/?auth_key=" + auto_key + "&limit=50&"
        
        datos = get_data_api(guid_url)
        if datos:
            datos_guid[guid] = pd.DataFrame(datos['result'][1:],columns=datos['result'][0])
    return datos_guid
#        pdb.set_trace()

Adicionalmente se ubicaron guids dentro de la API de forma Manual para complementar la data. Estos se agregan manualmente y realiza una validación para evitar duplicados.

In [32]:
guids_name = guids['guid']
guids_name = guids_name.to_list()
guids_name.extend(['PENET-DE-INTER-FIJO-57760','PENET-POR-HOGAR-NACIO-DEL','TOTAL-NACIO-DE-ACCES-48866','BANDA-ANCHA-Y-BANDA-ANGOS',
'SERIE-TRIME-DE-ACCES-A','ACCES-A-INTER-FIJO-POR','DISTR-DE-LOS-ACCES-76145','ACCES-A-INTER-FIJO-23248','ACCES-A-INTER-FIJO-97624',
'ACCES-A-INTER-FIJO-16249','ACCES-A-INTER-FIJO-62463'])
guids_name = list(set(guids_name))

In [33]:
datos_guid = guids_reader(guids_name)

Error al realizar la solicitud: Código de estado 500
Url http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/SERIE-TRIME-DE-ACCES-A/data.ajson/?auth_key=ZXuPjCTsAAlw8wdgE6mXkm66ZfAi9kO9YXO4WeV3&limit=50&
Error al realizar la solicitud: Código de estado 500
Url http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/DISTR-DE-LOS-ACCES-76145/data.ajson/?auth_key=ZXuPjCTsAAlw8wdgE6mXkm66ZfAi9kO9YXO4WeV3&limit=50&
Error al realizar la solicitud: Código de estado 500
Url http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/PENET-DE-INTER-FIJO-57760/data.ajson/?auth_key=ZXuPjCTsAAlw8wdgE6mXkm66ZfAi9kO9YXO4WeV3&limit=50&
Error al realizar la solicitud: Código de estado 500
Url http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/PENET-POR-HOGAR-NACIO-DEL/data.ajson/?auth_key=ZXuPjCTsAAlw8wdgE6mXkm66ZfAi9kO9YXO4WeV3&limit=50&
Error al realizar la solicitud: Código de estado 500
Url http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/PENET-DEL-SERVI-DE-INTER/data.ajson/?

Adicionalmente, sera útil para los fines de análisis los datos de localidades y provincias:

In [34]:
# Datos de Localidades
url = 'https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.5/download/localidades.json'
localidades = get_data_api(url)

# Desanidado de columnas

for index, data_localidad in enumerate(localidades['localidades']):
    localidades['localidades'][index]['latitud'] = data_localidad['centroide']['lat']
    localidades['localidades'][index]['longitud'] = data_localidad['centroide']['lon']
    localidades['localidades'][index]['provincia_nombre'] = data_localidad['provincia']['nombre']
    localidades['localidades'][index]['provincia_id'] = data_localidad['provincia']['id']
    localidades['localidades'][index]['municipio_nombre'] = data_localidad['municipio']['nombre']
    localidades['localidades'][index]['municipio_id'] = data_localidad['municipio']['id']
    localidades['localidades'][index]['departamento_nombre'] = data_localidad['departamento']['nombre']
    localidades['localidades'][index]['departamento_id'] = data_localidad['departamento']['id']
    localidades['localidades'][index]['localidad_censal_nombre'] = data_localidad['localidad_censal']['nombre']
    localidades['localidades'][index]['localidad_censal_id'] = data_localidad['localidad_censal']['id']

datos_guid['Localidades'] = pd.DataFrame.from_dict(localidades['localidades'])
datos_guid['Localidades'].drop(columns = ['centroide','provincia','municipio','departamento','localidad_censal'],inplace = True)
datos_guid['Localidades'].head()

,categoria,fuente,nombre,id,latitud,longitud,provincia_nombre,provincia_id,municipio_nombre,municipio_id,departamento_nombre,departamento_id,localidad_censal_nombre,localidad_censal_id
0,Localidad simple,INDEC,ALBERTI,06021010000,-35.033073,-60.280620,Buenos Aires,06,Alberti,060021,Alberti,06021,Alberti,06021010
1,Localidad simple,INDEC,CORONEL SEGUI,06021020000,-34.868119,-60.393971,Buenos Aires,06,Alberti,060021,Alberti,06021,Coronel Seguí,06021020
2,Componente de localidad compuesta,INDEC,MECHITA,06021030000,-35.068014,-60.402597,Buenos Aires,06,Alberti,060021,Alberti,06021,Mechita,06021030
3,Localidad simple,INDEC,PLA,06021040000,-35.124382,-60.220061,Buenos Aires,06,Alberti,060021,Alberti,06021,Pla,06021040
4,Localidad simple,INDEC,VILLA GRISOLIA,06021050000,-35.109618,-60.070551,Buenos Aires,06,Alberti,060021,Alberti,06021,Villa Grisolía,06021050


In [35]:
# Ejemplo de acceso
datos_guid['Localidades'][datos_guid['Localidades']['localidad_censal_id'] == '0'+'6035010']

,categoria,fuente,nombre,id,latitud,longitud,provincia_nombre,provincia_id,municipio_nombre,municipio_id,departamento_nombre,departamento_id,localidad_censal_nombre,localidad_censal_id
19,Entidad,INDEC,AREA CINTURON ECOLOGICO,06035010001,-34.675006,-58.310563,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
20,Entidad,INDEC,AVELLANEDA,06035010002,-34.662651,-58.365699,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
21,Entidad,INDEC,CRUCESITA,06035010003,-34.669106,-58.357454,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
22,Entidad,INDEC,DOCK SUD,06035010004,-34.649127,-58.341177,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
23,Entidad,INDEC,GERLI,06035010005,-34.684764,-58.372603,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
24,Entidad,INDEC,PIÑEYRO,06035010006,-34.668550,-58.388603,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
25,Entidad,INDEC,SARANDI,06035010007,-34.684372,-58.348101,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
26,Entidad,INDEC,VILLA DOMINICO,06035010008,-34.698632,-58.336360,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010
27,Entidad,INDEC,WILDE,06035010009,-34.706159,-58.321835,Buenos Aires,06,Avellaneda,060035,Avellaneda,06035,Avellaneda,06035010


In [36]:
# Datos de Provincias
url = 'https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.2/download/provincias.json'
provincias = get_data_api(url)
# Desanidado de columnas

for index, data_provincias in enumerate(provincias['provincias']):
    provincias['provincias'][index]['latitud'] = data_provincias['centroide']['lat']
    provincias['provincias'][index]['longitud'] = data_provincias['centroide']['lon']

datos_guid['Provincias'] = pd.DataFrame.from_dict(provincias['provincias'])
datos_guid['Provincias'].drop(columns = ['nombre_completo','fuente','iso_id','categoria','iso_nombre','centroide'],inplace = True)
datos_guid['Provincias'].head()


,nombre,id,latitud,longitud
0,Misiones,54,-26.875397,-54.651697
1,San Luis,74,-33.757726,-66.028130
2,San Juan,70,-30.865368,-68.889491
3,Entre Ríos,30,-32.058874,-59.201448
4,Santa Cruz,78,-48.815485,-69.955762


Se muestran los guids con datos:

In [37]:
datos_guid.keys()

dict_keys(['CONEC-AL-SERVI-DE-INTER', 'BANDA-ANCHA-Y-BANDA-ANGOS', 'ACCES-A-INTER-FIJO-23248', 'ACCES-A-INTER-FIJO-POR', 'VELOC-PROME-DE-BAJAD-DE', 'ACCES-A-INTER-FIJO-97624', 'INGRE-POR-LA-OPERA-DEL', 'ACCES-A-INTER-FIJO-62463', 'ACCES-A-INTER-FIJO-16249', 'TOTAL-NACIO-DE-ACCES-48866', 'Localidades', 'Provincias'])

In [41]:
# Ejemplo de como acceder a un GUID 
datos_guid['CONEC-AL-SERVI-DE-INTER'].head()

,Provincia,Partido,Localidad,Población,ADSL,Cablemódem,Dial Up,Fibra óptica,Satelital,Wireless,Telefonía Fija,3G,4G,Link,Latitud,Longitud
0,BUENOS AIRES,9 de Julio,Alfredo Demarchi (Est. Facundo Quiroga),1853,SI,--,--,--,--,--,SI,SI,SI,6588010,"-35,29298195","-61,40725421"
1,BUENOS AIRES,9 de Julio,Carlos María Naón,497,--,--,--,--,--,--,SI,SI,SI,6588020,"-35,23949938","-60,82513588"
2,BUENOS AIRES,9 de Julio,Dudignac,2670,--,--,SI,--,--,SI,SI,SI,SI,6588040,"-35,65119285","-60,70989628"
3,BUENOS AIRES,9 de Julio,La Aurora (Est. La Niña),531,--,--,--,--,--,--,--,--,SI,6588050,"-35,40794679","-61,21056682"
4,BUENOS AIRES,9 de Julio,Manuel B. Gonnet (Est. French),748,SI,--,--,SI,--,--,SI,SI,SI,6588060,"-35,52014145","-60,9989488"


Se guardan los dataframes generados:

In [39]:
for dataframe in datos_guid.keys():
    datos_guid[dataframe].to_parquet('..\\datasets\\Fuente\\' + dataframe + '.parquet',index=False)